In [ ]:
%matplotlib notebook
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '' # Use CPU

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K


from abyss_deep_learning.keras.activations import Hexpo

In [ ]:
a, b, c, d = 1, 1, 1, 1
input_shape = (100, 1)
x_data = np.linspace(-2.5, 2.5, input_shape[0])[..., np.newaxis]

with K.tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
    x = K.tf.placeholder(K.tf.float32, shape=((None,) + input_shape[1:]),name='x')
    feed_dict = {x: x_data}
    hexpo_y = Hexpo._activation(x, a, b, c, d)
    hexpo_dy = K.tf.gradients(hexpo_y, x)
    result = sess.run([hexpo_y, hexpo_dy], feed_dict=feed_dict)

plt.figure()
plt.subplot(1, 2, 1)
plt.plot(feed_dict[x], result[0])
plt.subplot(1, 2, 2)
plt.semilogy(feed_dict[x].squeeze(), np.array(result[1]).squeeze(), 'r')

In [ ]:
N = 1000
input_shape = (48, 64, 1)
func = lambda x: np.sin(x) + 1
x_train = np.random.uniform(-1, 1, size=((N,) + input_shape))
x_test = np.random.uniform(-1, 1, size=((N,) + input_shape))
y_train = func(x_train)
y_test = func(x_test)
print(x_train.shape)

In [ ]:
from keras.layers.advanced_activations import PReLU
def make_model(input_shape):
    from keras.layers import Input, Dense
    from keras.models import Model
    a, b, c, d = 1, 1, 1, 1
    x = Input(shape=input_shape)
    y = Hexpo(shared_axes=(1, 2))(Dense(1)(x))
    y = Hexpo(shared_axes=(1, 2))(Dense(1)(y))
    return Model(x, y)

model = make_model(input_shape)
model.compile(loss='mse', optimizer='nadam')
model.fit(x_train, y_train, verbose=1, epochs=10, validation_split=0.1)
y_pred = model.predict(x_test, batch_size=100)
# y_train
# plt.figure()
# plt.subplot(1, 2, 1)
# plt.plot(feed_dict[x], result[0])
# plt.subplot(1, 2, 2)
# plt.semilogy(feed_dict[x].squeeze(), np.array(result[1]).squeeze(), 'r')
model.summary()

In [ ]:
# print(model.get_config())

for w in model.get_layer(name='hexpo_1').trainable_weights:
    print(K.eval(w))
    
model.get_layer(name='hexpo_1').get_config()